## baselinemodel

In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import models, layers, optimizers

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import OneHotEncoder

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
os.chdir("/content/gdrive/MyDrive/smArt")

In [4]:
os.listdir()

['Fine-art Painting Classification.gslides',
 'wikiart',
 'whole_set.csv',
 'image_arrays_again.npy',
 'balanced_set_final.csv',
 'balanced_arrays_2966.npy']

In [5]:
df = pd.read_csv("balanced_set_final.csv")

In [6]:
array = np.load("balanced_arrays_2966.npy")

In [7]:
def load_model():
    
    # $CHALLENGIFY_BEGIN
    
    model = VGG16(weights="imagenet", include_top=False, input_shape=(128,128,3))
    
    # $CHALLENGIFY_END
    
    return model

In [8]:
def set_nontrainable_layers(model):
    
    # $CHALLENGIFY_BEGIN
    # Set the first layers to be untrainable
    model.trainable = False
    
    # $CHALLENGIFY_END
    
    return model

In [9]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    # $CHALLENGIFY_BEGIN
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    dense_layer_2 = layers.Dense(100, activation='relu')
    prediction_layer = layers.Dense(15, activation='softmax')
    
    
    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        dense_layer_2,
        prediction_layer
    ])
    # $CHALLENGIFY_END
    return model


In [10]:
def build_model():
    # $CHALLENGIFY_BEGIN    
    model = load_model()
    model = add_last_layers(model)
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model
    # $CHALLENGIFY_END

In [11]:
df["genre"].unique()

array(['Abstract_Expressionism', 'Art_Nouveau_Modern', 'Baroque',
       'Color_Field_Painting', 'Cubism', 'Expressionism', 'Impressionism',
       'Naive_Art_Primitivism', 'Northern_Renaissance', 'Pop_Art',
       'Post_Impressionism', 'Realism', 'Rococo', 'Romanticism',
       'Symbolism'], dtype=object)

In [12]:
y = df["genre"]

In [13]:
def genre_to_num(x):
  genres = ['Abstract_Expressionism', 'Art_Nouveau_Modern', 'Baroque',
       'Color_Field_Painting', 'Cubism', 'Expressionism', 'Impressionism',
       'Naive_Art_Primitivism', 'Northern_Renaissance', 'Pop_Art',
       'Post_Impressionism', 'Realism', 'Rococo', 'Romanticism',
       'Symbolism']
  return genres.index(x)


In [14]:
y = y.apply(genre_to_num)

In [15]:
len(y.value_counts())

15

In [16]:
X = array

In [17]:
X.shape

(44490, 128, 128, 3)

In [18]:
y.shape

(44490,)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [20]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

In [21]:
from tensorflow.keras.utils import to_categorical

# $CHALLENGIFY_BEGIN
y_train_cat = to_categorical(y_train, num_classes=15)
y_test_cat = to_categorical(y_test, num_classes=15)
y_val_cat = to_categorical(y_val, num_classes =15)
# $CHALLENGIFY_END

In [22]:
X_train.shape

(23357, 128, 128, 3)

In [23]:
y_train_cat.shape

(23357, 15)

In [24]:
X_val.shape

(7786, 128, 128, 3)

In [25]:
y_val_cat.shape

(7786, 15)

In [26]:
model = build_model()
model.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 500)               4096500   
                                                                 
 dense_1 (Dense)             (None, 100)               50100     
                                                                 
 dense_2 (Dense)             (None, 15)                1515      
                                                                 
Total params: 18,862,803
Trainable params: 4,148,115
Non-trainable params: 14,714,688
_____________________________________

In [27]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_model()

es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 5, 
                   verbose = 1, 
                   restore_best_weights = True)

history = model.fit(X_train, y_train_cat, 
                    validation_data=(X_val, y_val_cat), 
                    epochs=50, 
                    batch_size=16, 
                    callbacks=[es])

Epoch 1/50
1460/1460 [==============================] - 69s 40ms/step - loss: 2.7603 - accuracy: 0.2081 - val_loss: 2.2298 - val_accuracy: 0.2701
Epoch 2/50
1460/1460 [==============================] - 57s 39ms/step - loss: 1.9406 - accuracy: 0.3570 - val_loss: 2.1370 - val_accuracy: 0.3326
Epoch 3/50
1460/1460 [==============================] - 58s 39ms/step - loss: 1.5424 - accuracy: 0.4739 - val_loss: 2.1524 - val_accuracy: 0.3567
Epoch 4/50
1460/1460 [==============================] - 57s 39ms/step - loss: 1.2160 - accuracy: 0.5789 - val_loss: 2.2054 - val_accuracy: 0.3642
Epoch 5/50
1460/1460 [==============================] - 65s 44ms/step - loss: 0.9280 - accuracy: 0.6744 - val_loss: 2.3586 - val_accuracy: 0.3911
Epoch 6/50
1460/1460 [==============================] - 65s 44ms/step - loss: 0.6982 - accuracy: 0.7551 - val_loss: 2.6469 - val_accuracy: 0.3892
Epoch 7/50
1460/1460 [==============================] - 58s 39ms/step - loss: 0.5290 - accuracy: 0.8180 - val_loss: 2.9444 -